# TensorFlow Dataset API

**Learning Objectives**
1. Learn how use tf.data to read data from memory
1. Learn how to use tf.data in a training loop
1. Learn how use tf.data to read data from disk
1. Learn how to write production input pipelines with feature engineering (batching, shuffling, etc.)


In this notebook, we will start by refactoring the linear regression we implemented in the previous lab so that is takes its data from a`tf.data.Dataset`, and we will learn how to implement **stochastic gradient descent** with it. In this case, the original dataset will be synthetic and read by the `tf.data` API directly  **from memory.**

In a second part, we will learn how to load a dataset with the `tf.data` API when the dataset **resides on disk.**

In [1]:
import json
import math
import os
from pprint import pprint

import numpy as np
import tensorflow as tf

print(tf.version.VERSION)

2.8.0


## Loading data from memory

### Creating the dataset

Let's consider the synthetic dataset of the previous section:

In [2]:
N_POINTS = 10
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10

2022-04-04 15:38:41.096930: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-04-04 15:38:41.097034: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-04 15:38:41.097070: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-6): /proc/driver/nvidia/version does not exist
2022-04-04 15:38:41.109839: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])

print("Dataset:", type(dataset) )
for elem in dataset:
    print(elem.numpy())

it = iter(dataset)
print("Iterator")
print(next(it).numpy())
print(next(it).numpy())

print("Reduce dataset:")
print(dataset.reduce(0, lambda state, value: state + value).numpy())

Dataset: <class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>
8
3
0
8
2
1
Iterator
8
3
Reduce dataset:
22


We begin with implementing a function that takes as input


- our $X$ and $Y$ vectors of synthetic data generated by the linear function $y= 2x + 10$
- the number of passes over the dataset we want to train on (`epochs`)
- the size of the batches the dataset (`batch_size`)

and returns a `tf.data.Dataset`: 

**Remark:** Note that the last batch may not contain the exact number of elements you specified because the dataset was exhausted.

If you want batches with the exact same number of elements per batch, we will have to discard the last batch by
setting:

```python
dataset = dataset.batch(batch_size, drop_remainder=True)
```

We will do that here.

**Lab Task #1:** Complete the code below to 
1. instantiate a `tf.data` dataset using [tf.data.Dataset.from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices).
2. Set up the dataset to 
  * repeat `epochs` times,
  * create a batch of size `batch_size`, ignoring extra elements when the batch does not divide the number of input elements evenly.

In [4]:
def create_dataset(X, Y, epochs, batch_size):  # X and Y are tf.constant
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))  # Creates a `Dataset` whose elements are slices of the given tensors.
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)  # Combines consecutive elements of this dataset into batches.
    return dataset

Let's test our function by iterating twice over our dataset in batches of 3 datapoints:

In [5]:
BATCH_SIZE = 3
EPOCH      = 2

dataset = create_dataset(X, Y, epochs=EPOCH, batch_size=BATCH_SIZE)
for i, (x, y) in enumerate(dataset):
    print("x:", x.numpy(), "y:", y.numpy())
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE

assert EPOCH

x: [0. 1. 2.] y: [10. 12. 14.]
x: [3. 4. 5.] y: [16. 18. 20.]
x: [6. 7. 8.] y: [22. 24. 26.]
x: [9. 0. 1.] y: [28. 10. 12.]
x: [2. 3. 4.] y: [14. 16. 18.]
x: [5. 6. 7.] y: [20. 22. 24.]


### Loss function and gradients

The loss function and the function that computes the gradients are the same as before:

In [6]:
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y) ** 2
    return tf.reduce_mean(errors)


def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

### Training loop

The main difference now is that now, in the traning loop, we will iterate directly on the `tf.data.Dataset` generated by our `create_dataset` function. 

We will configure the dataset so that it iterates 250 times over our synthetic dataset in batches of 2.

**Lab Task #2:** Complete the code in the cell below to call your dataset above when training the model. Note that the `step, (X_batch, Y_batch)` iterates over the `dataset`. The inside of the `for` loop should be exactly as in the previous lab. 

In [7]:
EPOCHS = 300
BATCH_SIZE = 2
LEARNING_RATE = 0.01

MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

# X, Y, epochs=1, batch_size=3
dataset =  create_dataset(X,Y, epochs=EPOCHS, batch_size=BATCH_SIZE) # TODO: Your code here.
for step, (X_batch, Y_batch) in enumerate(dataset): # TODO -- Your code here.
    dw0, dw1 =  compute_gradients(X_batch, Y_batch, w0, w1)
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)

    if step % 100 == 0:
        loss =  loss_mse(X_batch, Y_batch, w0, w1)  # TODO -- Your code here.
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))

assert loss < 0.0001
assert abs(w0 - 2) < 0.001
assert abs(w1 - 10) < 0.001

X shape: (10,)
Y shape: (10,)
STEP 0 - loss: 115.80199432373047, w0: 0.11999999731779099, w1: 0.2199999988079071

STEP 100 - loss: 22.102718353271484, w0: 2.5470058917999268, w1: 5.033106803894043

STEP 200 - loss: 6.4590983390808105, w0: 2.2957024574279785, w1: 7.314977645874023

STEP 300 - loss: 1.8875508308410645, w0: 2.1598522663116455, w1: 8.548519134521484

STEP 400 - loss: 0.5516019463539124, w0: 2.086413621902466, w1: 9.215352058410645

STEP 500 - loss: 0.16119401156902313, w0: 2.0467135906219482, w1: 9.575833320617676

STEP 600 - loss: 0.04710635915398598, w0: 2.0252528190612793, w1: 9.77070140838623

STEP 700 - loss: 0.013765865936875343, w0: 2.0136513710021973, w1: 9.876045227050781

STEP 800 - loss: 0.004022936336696148, w0: 2.0073795318603516, w1: 9.932991027832031

STEP 900 - loss: 0.0011757200118154287, w0: 2.0039892196655273, w1: 9.963774681091309

STEP 1000 - loss: 0.0003435480175539851, w0: 2.0021562576293945, w1: 9.98041820526123

STEP 1100 - loss: 0.0001003441138891

## Loading data from disk

### Locating the CSV files

We will start with the **taxifare dataset** CSV files that we wrote out in a previous lab. 

The taxifare datast files been saved into `../data`.

Check that it is the case in the cell below, and, if not, regenerate the taxifare
dataset by running the provious lab notebook:

In [8]:
!ls -l ../data/taxi*.csv

-rw-r--r-- 1 jupyter jupyter 123590 Mar 15 10:40 ../data/taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 579055 Mar 15 10:40 ../data/taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 123114 Mar 15 10:40 ../data/taxi-valid.csv


### Use tf.data to read the CSV files

The `tf.data` API can easily read csv files using the helper function

[tf.data.experimental.make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)


If you have TFRecords (which is recommended), you may use

[tf.data.experimental.make_batched_features_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_batched_features_dataset)

The first step is to define 

- the feature names into a list `CSV_COLUMNS`
- their default values into a list `DEFAULTS`

In [9]:
CSV_COLUMNS = [
    "fare_amount",
    "pickup_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "passenger_count",
    "key",
]
LABEL_COLUMN = "fare_amount"
DEFAULTS = [[0.0], ["na"], [0.0], [0.0], [0.0], [0.0], [0.0], ["na"]]

Let's now wrap the call to `make_csv_dataset` into its own function that will take only the file pattern (i.e. glob) where the dataset files are to be located:

**Lab Task #3:** Complete the code in the `create_dataset(...)` function below to return a `tf.data` dataset made from the `make_csv_dataset`. Have a look at the [documentation here](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset). The `pattern` will be given as an argument of the function but you should set the `batch_size`, `column_names` and `column_defaults`.

In [10]:
# def create_dataset(X, Y, epochs, batch_size):  # X and Y are tf.constant
#     dataset = tf.data.Dataset.from_tensor_slices((X,Y))  # Creates a `Dataset` whose elements are slices of the given tensors.
#     dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)  # Combines consecutive elements of this dataset into batches.
#     return dataset

In [20]:
def create_dataset(pattern):
    # TODO: Your code here.
    return tf.data.experimental.make_csv_dataset(
        pattern, batch_size=1, column_names=CSV_COLUMNS, column_defaults=DEFAULTS
    )

tempds = create_dataset("../data/taxi-train*")
print(tempds)

<PrefetchDataset element_spec=OrderedDict([('fare_amount', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('pickup_datetime', TensorSpec(shape=(1,), dtype=tf.string, name=None)), ('pickup_longitude', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('pickup_latitude', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('dropoff_longitude', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('dropoff_latitude', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('passenger_count', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('key', TensorSpec(shape=(1,), dtype=tf.string, name=None))])>


Note that this is a prefetched dataset, where each element is an `OrderedDict` whose keys are the feature names and whose values are tensors of shape `(1,)` (i.e. vectors).

Let's iterate over the two first element of this dataset using `dataset.take(2)` and let's convert them ordinary Python dictionary with numpy array as values for more readability:

In [23]:
for data in tempds.take(2):
    pprint({k: v.numpy() for k, v in data.items()})
    print("\n")

{'dropoff_latitude': array([40.73161], dtype=float32),
 'dropoff_longitude': array([-73.994705], dtype=float32),
 'fare_amount': array([3.7], dtype=float32),
 'key': array([b'4948'], dtype=object),
 'passenger_count': array([1.], dtype=float32),
 'pickup_datetime': array([b'2011-03-16 23:28:36 UTC'], dtype=object),
 'pickup_latitude': array([40.735535], dtype=float32),
 'pickup_longitude': array([-74.00179], dtype=float32)}


{'dropoff_latitude': array([40.713955], dtype=float32),
 'dropoff_longitude': array([-74.01029], dtype=float32),
 'fare_amount': array([6.1], dtype=float32),
 'key': array([b'4895'], dtype=object),
 'passenger_count': array([3.], dtype=float32),
 'pickup_datetime': array([b'2010-02-25 20:14:00 UTC'], dtype=object),
 'pickup_latitude': array([40.724567], dtype=float32),
 'pickup_longitude': array([-73.998436], dtype=float32)}




### Transforming the features

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary:

1. Remove the unwanted column "key"
1. Keep the label separate from the features

Let's first implement a funciton that takes as input a row (represented as an `OrderedDict` in our `tf.data.Dataset` as above) and then returns a tuple with two elements:

* The first element beeing the same `OrderedDict` with the label dropped
* The second element beeing the label itself (`fare_amount`)

Note that we will need to also remove the `key` and `pickup_datetime` column, which we won't use.

**Lab Task #4a:** Complete the code in the `features_and_labels(...)` function below. Your function should return a dictionary of features and a label. Keep in mind `row_data` is already a dictionary and you will need to remove the `pickup_datetime` and `key` from `row_data` as well.

In [24]:
# "fare_amount",
#     "pickup_datetime",
#     "pickup_longitude",
#     "pickup_latitude",
#     "dropoff_longitude",
#     "dropoff_latitude",
#     "passenger_count",
#     "key",

In [25]:
UNWANTED_COLS = ["pickup_datetime", "key"]


# TODO 4a
def features_and_labels(row_data):
    label =  row_data.pop(LABEL_COLUMN) # TODO: Your code here.
    features =  row_data # TODO: Your code here.
    
    # TODO: Your code here.
    for col in UNWANTED_COLS:
        features.pop(col)
    return features, label

Let's iterate over 2 examples from our `tempds` dataset and apply our `feature_and_labels`
function to each of the examples to make sure it's working:

In [26]:
for row_data in tempds.take(2):
    features, label = features_and_labels(row_data)
    pprint(features)
    print(label, "\n")
    assert UNWANTED_COLS[0] not in features.keys()
    assert UNWANTED_COLS[1] not in features.keys()
    assert label.shape == [1]

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-74.004135], dtype=float32)>),
             ('pickup_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.742714], dtype=float32)>),
             ('dropoff_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-73.98735], dtype=float32)>),
             ('dropoff_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.77097], dtype=float32)>),
             ('passenger_count',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>)])
tf.Tensor([8.1], shape=(1,), dtype=float32) 

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-74.01203], dtype=float32)>),
             ('pickup_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.716934], dtype=float32)>),
             ('dropoff_longitude',
              <tf

### Batching

Let's now refactor our `create_dataset` function so that it takes an additional argument `batch_size` and batch the data correspondingly. We will also use the `features_and_labels` function we implemented in order for our dataset to produce tuples of features and labels.

**Lab Task #4b:** Complete the code in the `create_dataset(...)` function below to return a `tf.data` dataset made from the `make_csv_dataset`. Now, the `pattern` and `batch_size` will be given as an arguments of the function but you should set the `column_names` and `column_defaults` as before. You will also apply a `.map(...)` method to create features and labels from each example. 

In [28]:
def create_dataset(pattern, batch_size):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS
    )
    dataset = dataset.map(features_and_labels)   # TODO: Your code here.
    return dataset

Let's test that our batches are of the right size:

In [29]:
BATCH_SIZE = 2

tempds = create_dataset("../data/taxi-train*", batch_size=2)

for X_batch, Y_batch in tempds.take(2):
    pprint({k: v.numpy() for k, v in X_batch.items()})
    print(Y_batch.numpy(), "\n")
    assert len(Y_batch) == BATCH_SIZE

{'dropoff_latitude': array([40.77929 , 40.732643], dtype=float32),
 'dropoff_longitude': array([-73.96116, -73.9855 ], dtype=float32),
 'passenger_count': array([1., 1.], dtype=float32),
 'pickup_latitude': array([40.74611 , 40.723167], dtype=float32),
 'pickup_longitude': array([-73.98201, -73.99684], dtype=float32)}
[11.   5.3] 

{'dropoff_latitude': array([40.68153 , 40.778027], dtype=float32),
 'dropoff_longitude': array([-73.99634 , -73.981895], dtype=float32),
 'passenger_count': array([2., 1.], dtype=float32),
 'pickup_latitude': array([40.754524, 40.761623], dtype=float32),
 'pickup_longitude': array([-73.97805 , -73.998184], dtype=float32)}
[20.5  8.1] 



### Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

Let's refactor our `create_dataset` function so that it shuffles the data, when the dataset is used for training.

We will introduce a additional argument `mode` to our function to allow the function body to distinguish the case
when it needs to shuffle the data (`mode == "train"`) from when it shouldn't (`mode == "eval"`).

Also, before returning we will want to prefetch 1 data point ahead of time (`dataset.prefetch(1)`) to speedup training:

**Lab Task #4c:** The last step of our `tf.data` dataset will specify shuffling and repeating of our dataset pipeline. Complete the code below to add these three steps to the Dataset pipeline
1. follow the `.map(...)` operation which extracts features and labels with a call to `.cache()` the result.
2. during training, use `.shuffle(...)` and `.repeat()` to shuffle batches and repeat the dataset
3. use `.prefetch(...)` to take advantage of multi-threading and speedup training.

In [30]:
def create_dataset(pattern, batch_size=1, mode="eval"):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS
    )

    dataset = dataset.map(features_and_labels).cache()    # TODO: Your code here.

    if mode == 'train':
        dataset = dataset.shuffle(1000).repeat()   # TODO: Your code here.

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)  # TODO: Your code here.

    return dataset

Let's check that our function work well in both modes:

In [31]:
tempds = create_dataset("../data/taxi-train*", 2, "train")
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.962616, -73.98169 ], dtype=float32)>), ('pickup_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.77303, 40.73264], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.97303, -73.91471], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.764137, 40.76598 ], dtype=float32)>), ('passenger_count', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 5.5, 30. ], dtype=float32)>)]


2022-04-04 17:03:21.112209: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [32]:
tempds = create_dataset("../data/taxi-valid*", 2, "eval")
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-74.01481, -73.97369], dtype=float32)>), ('pickup_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.709534, 40.753525], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.95427, -73.96727], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.771374, 40.760807], dtype=float32)>), ('passenger_count', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: shape=(2,), dtype=float32, numpy=array([23.5,  6.5], dtype=float32)>)]


2022-04-04 17:03:21.393215: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In the next notebook, we will build the model using this input pipeline.

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.